## Data preparation

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# # Load the data
# file_path = "../raw_data/all_expanded_ML.csv"
# df = pd.read_csv(file_path)

# # Drop unnecessary columns
# columns_to_drop = ['Região', 'Município', 'Estado', 'Código_UF', 'Magreza_total_%', 'UF']
# df_dropped = df.drop(columns=columns_to_drop)

# # Filter based on 'Localização' and create separate datasets for urban and rural
# df_urban = df_dropped[df_dropped['Localização'] == 'Urbana'].drop(columns=['Localização'])
# df_rural = df_dropped[df_dropped['Localização'] == 'Rural'].drop(columns=['Localização'])

# df_urban.to_csv("../raw_data/all_urban_expanded_ML.csv")
# df_rural.to_csv("../raw_data/all_rural_expanded_ML.csv")

# DL training workflow

## Load the Dataset

In [ ]:
df_urban = pd.read_csv("../raw_data/all_urban_expanded_ML.csv")

## Create sequences 

In [ ]:
# Sort by 'Ano' and 'Código_IBGE' and group by 'Código_IBGE'
df_urban_sorted = df_urban.sort_values(by=['Código_IBGE', 'Ano'])
grouped_urban = [group for _, group in df_urban_sorted.groupby('Código_IBGE')]

# Filter each group to include data from 2012 to 2020 and create sequences
def filter_and_create_sequences(grouped_data):
    sequences = []
    targets = []
    for group in grouped_data:
        filtered_group = group[(group['Ano'] >= 2012) & (group['Ano'] <= 2020)]
        if len(filtered_group) == 9:
            sequence = filtered_group.drop(columns=['Ano', 'Código_IBGE', 'Adjusted_funding']).values
            target_values = filtered_group['Adjusted_funding'].values
            sequences.append(sequence)
            targets.append(target_values)
    return np.array(sequences), np.array(targets)


# Create sequences and targets for urban datasets
array_urban, y_urban = filter_and_create_sequences(grouped_urban)

## Create Test and Train sets

In [ ]:
# Split data into training and testing sets
def split_data(features, targets):
    X_train = features[:, :6, :]
    X_test = features[:, 6:, :]
    y_train = targets[:, 6:]
    y_test = targets[:, 6:]
    return X_train, X_test, y_train, y_test

# Split the data for both urban datasets
X_train_urban, X_test_urban, y_train_urban, y_test_urban = split_data(array_urban, y_urban)


## Train and evaluate model

### Working LSTM Code

In [ ]:
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import SimpleRNN, Dense, Masking, LSTM, Dropout
from tensorflow.keras.callbacks import EarlyStopping
 from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.regularizers import l2

# Scale the features
feature_scaler = MinMaxScaler(feature_range=(0, 1))

# Flatten the 3D array to 2D for scaling, then reshape back to 3D
X_train_urban_flatten = X_train_urban.reshape(-1, X_train_urban.shape[-1])
X_train_urban_scaled = feature_scaler.fit_transform(X_train_urban_flatten).reshape(X_train_urban.shape)

X_test_urban_flatten = X_test_urban.reshape(-1, X_test_urban.shape[-1])
X_test_urban_scaled = feature_scaler.transform(X_test_urban_flatten).reshape(X_test_urban.shape)

# Scale the target variable
target_scaler = MinMaxScaler(feature_range=(0, 1))
y_train_urban_scaled = target_scaler.fit_transform(y_train_urban)
y_test_urban_scaled = target_scaler.transform(y_test_urban)

X_train_urban_scaled_paded = pad_sequences(X_train_urban_scaled, dtype='float32', padding='post', value=-1000)
# X_train_urban_scaled_paded.shape


# Build and compile the model
model = Sequential()
model.add(Masking(mask_value=-1000, input_shape=(None, 12)))

# First LSTM layer with L2 regularization
model.add(LSTM(units=20, activation='tanh', return_sequences=True, input_shape=(X_train_urban.shape[1], X_train_urban.shape[2]),
               kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(0.2))

# Second LSTM layer with L2 regularization
model.add(LSTM(units=20, activation='tanh',
               kernel_regularizer=l2(0.01), recurrent_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(0.2))

# Fully connected layers with L2 regularization
model.add(Dense(10, activation="relu", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))
model.add(Dropout(0.2))

model.add(Dense(3, activation="linear", kernel_regularizer=l2(0.01), bias_regularizer=l2(0.01)))


model.compile(loss='mse', optimizer='rmsprop', metrics=['mae'])

# Early stopping callback
es = EarlyStopping(patience=5)

# Train the model
history = model.fit(
    X_train_urban_scaled,
    y_train_urban_scaled,
    validation_split=0.2,
    epochs=30,
    batch_size=16,
    verbose=1,
    callbacks=[es]
)

# Evaluate the model
model.evaluate(X_test_urban_scaled, y_test_urban_scaled)

from sklearn.metrics import mean_squared_error, mean_absolute_error

# 1. Make Predictions on the scaled test features
y_pred_scaled = model.predict(X_test_urban_scaled)

# 2. Inverse Transform Predictions to original scale
y_pred = target_scaler.inverse_transform(y_pred_scaled)

# 3. Inverse Transform Actual Values to original scale
y_test_original = target_scaler.inverse_transform(y_test_urban_scaled)

# 4. Calculate Metrics on original scale
mse_original = mean_squared_error(y_test_original, y_pred)
mae_original = mean_absolute_error(y_test_original, y_pred)

print(f"Mean Squared Error on original scale: {mse_original}")
print(f"Mean Absolute Error on original scale: {mae_original}")

## Save Model

In [ ]:
history.model.save("../models/model.h5")

# Prediction workflow

## Prepare data for prediction

In [ ]:
# Sample a specific municipality using its Código_IBGE from the original dataset
sample_municipality_code = df_urban['Código_IBGE'].iloc[0]  # Taking the first municipality's code for demonstration
sample_data = df_urban[df_urban['Código_IBGE'] == sample_municipality_code]

# Filter the sample data to only include records from 2012 to 2020 (the range used for training)
sample_data_filtered = sample_data[(sample_data['Ano'] >= 2012) & (sample_data['Ano'] <= 2020)]
sample_data_filtered.columns

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Scale the features
feature_scaler = MinMaxScaler(feature_range=(0, 1))
sample_features_scaled = feature_scaler.fit_transform(sample_data_filtered.drop(columns=['Ano', 'Código_IBGE', 'Adjusted_funding']))

## Make prediction

In [ ]:
from tensorflow.keras.models import load_model

# Load the pre-trained model
new_model = load_model("../models/model.h5")
new_model.summary()

In [ ]:
# Make predictions using the model
predictions_scaled = model.predict(sample_features_scaled[-3:,:].reshape(1, 3, -1))

# Inverse transform the scaled predictions to the original scale
target_scaler = MinMaxScaler().fit()  # Assuming y_train_urban is your training target data
predictions_original_scale = target_scaler.inverse_transform(predictions_scaled)
predictions_original_scale
# The predictions for the next three years are now stored in predictions_original_scale